Insper

# Aula 6 - DataFrames

In [ ]:
# Criar a sessao do Spark
from pyspark.sql import SparkSession
spark = SparkSession \
            .builder \
            .master("local[*]") \
            .appName("Michel_DF") \
            .getOrCreate()

In [ ]:
spark

In [ ]:
sc = spark.sparkContext

Até agora vimos trabalhando com a estrutura base do Spark, as RDDs. Contudo, essas estruturas são trabalhosas e requerem o tratamento manual de tipos de dados bem como a interpretação constante dos dados posicionais na RDD.

É possível simplificar nosso trabalho com o uso de camadas novas e mais elevadas do Spark.


Existe uma série de funções para a criação e a criação e manipulação destas novas estruturas de dados. Vamos começar investigando os DataFrames, inspirados na biblioteca Pandas do Python.


## createDataFrame()
Para criar um DataFrame a partir de uma RDD, podemos utilizar a função `createDataFrame(RDD)`. É importante notar que o ponto de entrada das APIs de dados estruturados não é mais o `sparkContext`, mas diretamente a `SparkSession` acessível a partir da nossa variável `spark`.

In [ ]:
dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]



In [ ]:
dept

[('Finance', 10), ('Marketing', 20), ('Sales', 30), ('IT', 40)]

In [ ]:
df = spark.createDataFrame(dept, ["Department", "ID"])

# Mostrar o DataFrame
df.show()

+----------+---+
|Department| ID|
+----------+---+
|   Finance| 10|
| Marketing| 20|
|     Sales| 30|
|        IT| 40|
+----------+---+



Verificamos que a variável é do tipo DataFrame com colunas com nome `_1, _2,` e diferentes tipos, já __inferidos__ pelo Spark na criação do DataFrame a partir da RDD.

## take()
Podemos então utilizar a função take para verificar nosso DataFrame.

É interessante observar que a função nos mostra uma RDD e não um DataFrame. Essa RDD é composta de objetos do tipo Row, ou então linha. Isso é, um DataFrame nada mais é do que uma RDD onde cada elemento é uma linha de uma tabela com suas diferentes colunas.

Diferentemente das RDDs padrões, contudo, os objetos Row grava consigo os nomes e os tipos das colunas e o Spark mantém a gestão garantindo que um mesmo DataFrame seja composto de linhas do mesmo tipo.

## Row()

Podemos manualmente criar uma linha, a partir da classe Row.

Para isso precisamos nomear as colunas e seus valores.

Mais tarde entenderemos como concatenar diferentes objetos Row em um DataFrame.

## asDict()

Podemos transformar objetos do tipo Row em dicionários no python.

Para isso, vamos inicialmente guardar duas linhas de nosso DataFrame.

Na sequencia utilizamos a função asDict()

Com isso podemos acessar os valores de cada coluna normalmente como um dicionário.

## RDD
O atributo rdd do DataFrame também nos dá acesso direto à RDD fundamental que constrói o DataFrame.

Nessa RDD podemos aplicar todas as transformações e ações que aprendemos, desde que respeitando o tipo do elemento Row.

# Acessando o DataFrame
A ideia de termos um DataFrame, contudo, é não precisarmos operar diretamente nas RDDs. Para isso existe um conjunto de funções de acesso direto ao DataFrame. Como estas funções operam intrinsecamente em RDDs, elas também são transformações e ações.


## show()
A ação show() nos permite olhar o DataFrame como uma tabela, muito mais próximo do Pandas DataFrame. Cuidado, contudo, com o desejo de olhar o DataFrame completo.

## printSchema()

A ação show() não nos mostra claramente qual é o tipo dos dados envolvidos (similar a dtypes). Para isso, podemos imprimir o Schema do DataFrame.

O Schema é a definição do tipo de dado de cada coluna. É importante notar que o Spark inferiu esses dados a partir da RDD. O Spark infere a partir do primeiro elemento nos dados. Portanto, é importante que na leitura de arquivos, caso a inferência seja utilizada, garantir que a primeira linha do arquivo possua o tipo correto.

## dtypes

Podemos também utilizar o atributo dtypes para mostrar o tipo de cada coluna.

In [ ]:
df.dtypes

[('Department', 'string'), ('ID', 'bigint')]

In [ ]:
type(df.printSchema())

root
 |-- Department: string (nullable = true)
 |-- ID: long (nullable = true)



NoneType

# Definindo o Schema
Idealmente, quando operando em grandes massas de dados, a inferência do tipo pela primeira linha não é adequada e utilizamos como boa prática a definição manual do schema. Para isso precisamos utilizar os tipos de dados nativos do Spark que são convertidos para e de tipos python pela API pyspark.

Para isso importamos os tipos do spark.

## StructType(), StructField(), IntegerType(), StringType(), FloatType()
Cada tipo de variável é na verdade um objeto de uma classe. Então acessamos estes objetos para instanciar e para identificar os tipos de cada variável.

O Schema de um DataFrame é definido como um objeto StructType (similar a uma lista ou tupla) com campos do tipo StructField(). Cada campo (StructField) possui um nome, um tipo (objeto do tipo no spark) e um flag se é permitida a existência de nulos.

Assim, podemos definir explicitametne um Schema para nosso DataFrame.

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, LongType
schema = StructType([
    StructField("Department", StringType(), True),
    StructField("ID", LongType(), True)
])

# Criar o DataFrame com o esquema definido
df_schema = spark.createDataFrame(dept, schema)
df_schema.show()

+----------+---+
|Department| ID|
+----------+---+
|   Finance| 10|
| Marketing| 20|
|     Sales| 30|
|        IT| 40|
+----------+---+



Na sequencia, atribuímos o nosso schema ao DataFrame na sua criação.

In [ ]:
df_schema.printSchema()

root
 |-- Department: string (nullable = true)
 |-- ID: long (nullable = true)



Podemos então olhar nosso DataFrame e verificar que as colunas agora vêm identificadas.

In [ ]:
df_schema.show()

+----------+---+
|Department| ID|
+----------+---+
|   Finance| 10|
| Marketing| 20|
|     Sales| 30|
|        IT| 40|
+----------+---+



Podemos também verificar o Schema do DataFrame com as características que definimos anteriormente.

In [ ]:
df_schema.columns


['Department', 'ID']

In [ ]:
df_schema.take(2)

[Row(Department='Finance', ID=10), Row(Department='Marketing', ID=20)]

# Operações básicas com colunas

A exemplo do Pandas DataFrame, temos à nossa disposição uma série de transformações e ações que nos permite operar sobre o Spark DataFrame. Veremos a seguir as operações em colunas.

## columns
O atributo columns retorna, a exemplo do Pandas, uma lista com o nome das colunas. Contudo, diferente do Pandas, não é possível sobrescrever este atributo diretamente.

## withColumnRenamed()
Para renomear uma coluna utilizamos a função withColumnRenamed(). Esta função opera em uma coluna por vez. Assim, podemos utilizar um laço para renomear todas as colunas.

In [ ]:
for old_col, new_col in zip(df_schema.columns, ['dept_name', 'dept_id']):
    df_schema = df_schema.withColumnRenamed(old_col, new_col)

In [ ]:
df_schema.show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+



## drop()
Podemos descartar colunas usando a função .drop()

In [ ]:
df_schema.drop('dept_id').show()

+---------+
|dept_name|
+---------+
|  Finance|
|Marketing|
|    Sales|
|       IT|
+---------+



## Acessando colunas
Para acessar as colunas de um DataFrame, podemos utilizar a mesma notação python. Observe, contudo, que as colunas são representadas como objetos e, assim, o acesso à coluna não retorna automaticamente os dados existentes nela.

In [ ]:
df_schema["dept_id"]

Column<'dept_id'>

## select()
Para acessar os dados de uma coluna específica, precisamos utilizar a transformação select() seguida de uma ação show(). Esta transformação é similiar ao SELECT em SQL.

In [ ]:
df_schema.select('dept_id').show()

+-------+
|dept_id|
+-------+
|     10|
|     20|
|     30|
|     40|
+-------+



## Case sensitivity
Observe que a exemplo do SQL, DataFrames em Spark não são naturalmente sensíveis ao caso.

# Operações básicas com linhas
Da mesma forma que temos operações com colunas, temos operações com linhas.

## limit()
A transformação limit(LIM) nos permite limitar um número de registros (linhas no DataFrame) a ser retornado. POdemos concatenar com .collect() e temos o DataFrame no formato de RDD.


In [ ]:
df_schema.limit(2).collect()

[Row(dept_name='Finance', dept_id=10), Row(dept_name='Marketing', dept_id=20)]

Com isso podemos reduzir o número de linhas do nosso DataFrame para aumentar a velocidade do nosso trabalho.

## filter() - filtrando linhas
A transformação filter() nos permite filtrar linhas com base em condições especificadas sobre colunas.

In [ ]:
df_schema.filter(df_schema.dept_name == 'Finance').show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
+---------+-------+



In [ ]:
df_schema.filter(df_schema.dept_id == 10).show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
+---------+-------+



## where() - filtrando linhas

Para manter a similaridade como o SQL, a transformação filter() também pode ser acessada através de seu apelido (alias) where().

In [ ]:
df_schema.where(df_schema.dept_name == 'Finance').show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
+---------+-------+



In [ ]:
df_schema.where("dept_name = 'Finance'").show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
+---------+-------+



## distinct() - filtrando valores distintos
Utilizamos a transformação distinct() para selecionar apenas os valores distintos de uma coluna específica.

In [ ]:
df_schema.select('dept_name').distinct().show()

+---------+
|dept_name|
+---------+
|  Finance|
|Marketing|
|    Sales|
|       IT|
+---------+



## union() - concatenando linhas
Para concatenar linhas no DataFrame, precisamos criar uma RDD com objetos do tipo Row e Schema idêntico.

In [ ]:
from pyspark.sql import Row

# Criar novas linhas usando Row
new_rows = [Row(dept_name="Production", dept_id=50), Row(dept_name="IT", dept_id=60)]


In [ ]:
rdd_row = sc.parallelize(new_rows)

Transformamos essa lista em uma RDD, usando parallelize(), da mesma forma que fizemos anteriormente.

In [ ]:
newDF = spark.createDataFrame(rdd_row,schema=schema)

Por fim, criamos o DataFrame, definindo o Schema.

In [ ]:
sdf1 = df_schema.union(newDF)

In [ ]:
sdf1.show()

+----------+-------+
| dept_name|dept_id|
+----------+-------+
|   Finance|     10|
| Marketing|     20|
|     Sales|     30|
|        IT|     40|
|Production|     50|
|        IT|     60|
+----------+-------+



Podemos então concatenar nosso novo DataFrame ao DataFrame antigo.

In [ ]:
# Selecionar valores distintos na coluna dept_name
sdf1.select("dept_name").distinct().show()


+----------+
| dept_name|
+----------+
|   Finance|
| Marketing|
|     Sales|
|        IT|
|Production|
+----------+



## orderBy() - ordenando linhas
Com a transformação orderBy() podemos ordenar as linhas do DataFrame. Para isso, precisamos utilizar os métodos .asc() ou .desc() na coluna chave, indicando a ordem desejada.

In [ ]:
sdf1.orderBy(sdf1.dept_id.asc()).show()

+----------+-------+
| dept_name|dept_id|
+----------+-------+
|   Finance|     10|
| Marketing|     20|
|     Sales|     30|
|        IT|     40|
|Production|     50|
|        IT|     60|
+----------+-------+



In [ ]:
sdf1.orderBy(sdf1.dept_id.desc()).show()

+----------+-------+
| dept_name|dept_id|
+----------+-------+
|        IT|     60|
|Production|     50|
|        IT|     40|
|     Sales|     30|
| Marketing|     20|
|   Finance|     10|
+----------+-------+



# pyspark.sql.functions
Existe uma série de funções que facilitam a manipulação de DataFrames. Elas estão disponíveis no pacote pyspark.sql.functions e podem ser investigadas na referência abaixo:

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#module-pyspark.sql.functions

Podemos importá-las todas através da linha de comando abaixo.

In [ ]:
from pyspark.sql import functions as sf

## lit() - criando colunas
Como visto anteriormente, o Spark possui tipos próprios de dados com correspondência com o Python. Se quisermos criar uma coluna constante, podemos evitar o trabalho de criar a coluna em Python e transformá-la, utilizando a função lit(val) que cria uma coluna de literais (constantes) val.

In [ ]:
sf.lit(5)

Column<'5'>

## withColumn() - adicionando colunas

Para isso, precisamos concatenar uma coluna de literais. Isso é feito com a transformação withColumn(nome, nova_col).

In [ ]:
sdf1.withColumn('new_col', sf.lit(5)).show()

+----------+-------+-------+
| dept_name|dept_id|new_col|
+----------+-------+-------+
|   Finance|     10|      5|
| Marketing|     20|      5|
|     Sales|     30|      5|
|        IT|     40|      5|
|Production|     50|      5|
|        IT|     60|      5|
+----------+-------+-------+



## col() - acessando colunas

Em alguns casos precisamos explicitar que desejamos acessar um objeto do tipo Column. Para isso temos à nossa disposição a função .col(nome).

In [ ]:
sdf1.select(sf.col('dept_name')).show()

+----------+
| dept_name|
+----------+
|   Finance|
| Marketing|
|     Sales|
|        IT|
|Production|
|        IT|
+----------+



Observe que temos, com isso, diferentes formas de acessar colunas no select. Em alguns casos específicos será necessário optar pela menção explícita ao tipo do objeto usando .col()

In [ ]:
sdf1.select(sdf1.dept_id,sf.col('dept_name')).show()

+-------+----------+
|dept_id| dept_name|
+-------+----------+
|     10|   Finance|
|     20| Marketing|
|     30|     Sales|
|     40|        IT|
|     50|Production|
|     60|        IT|
+-------+----------+



Podemos ainda acessar a RDD resultante da transformação.

## alias()

A exemplo do SQL, podemos utilizar o método .alias() de uma coluna específica para renomear colunas rapidamente.

In [ ]:
sdf1.select("*",sf.col("dept_id").alias("Indentificador")).show()

+----------+-------+--------------+
| dept_name|dept_id|Indentificador|
+----------+-------+--------------+
|   Finance|     10|            10|
| Marketing|     20|            20|
|     Sales|     30|            30|
|        IT|     40|            40|
|Production|     50|            50|
|        IT|     60|            60|
+----------+-------+--------------+



In [ ]:
sdf1.select("*",sdf1["dept_id"].alias("Indentificador")).show()

+----------+-------+--------------+
| dept_name|dept_id|Indentificador|
+----------+-------+--------------+
|   Finance|     10|            10|
| Marketing|     20|            20|
|     Sales|     30|            30|
|        IT|     40|            40|
|Production|     50|            50|
|        IT|     60|            60|
+----------+-------+--------------+



# Expressões
DataFrames são equivalentes de tabelas no Pandas e também de tabelas no SQL. Buscando manter compatibilidade com o SQL, expressões permitem que escrevamos algumas expressões simplificadas de SQL para operar no DataFrame.


## expr()
Para isso, utilizamos a função expr(). Podemos, por exemplo, renomear colunas.

In [ ]:
sdf1.select(sf.expr("dept_name as departamento")).show()

+------------+
|departamento|
+------------+
|     Finance|
|   Marketing|
|       Sales|
|          IT|
|  Production|
|          IT|
+------------+



Podemos também operar sobre colunas diretamente, a exemplo do SQL.

In [ ]:
sdf1.select("*",sf.expr("dept_id / 10").alias("novo_ID")).show()

+----------+-------+-------+
| dept_name|dept_id|novo_ID|
+----------+-------+-------+
|   Finance|     10|    1.0|
| Marketing|     20|    2.0|
|     Sales|     30|    3.0|
|        IT|     40|    4.0|
|Production|     50|    5.0|
|        IT|     60|    6.0|
+----------+-------+-------+



## selectExpr()

Como a combinação de .select() e .expr() é muito comum, o spark já nos fornece um atalho: selectExpr().

In [ ]:
sdf1.selectExpr("*", "dept_id / 10 as novo_ID").show()

+----------+-------+-------+
| dept_name|dept_id|novo_ID|
+----------+-------+-------+
|   Finance|     10|    1.0|
| Marketing|     20|    2.0|
|     Sales|     30|    3.0|
|        IT|     40|    4.0|
|Production|     50|    5.0|
|        IT|     60|    6.0|
+----------+-------+-------+



# Abrindo arquivos CSV
Até agora trabalhamos com DataFrames criados manualmente a partir de uma RDD prévia. Podemos utilizar as funções do pacote .read do Spark para ler diretamente fontes de dados. Para carregar arquivos .csv usamos a função .csv.


In [1]:
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip
!unzip ml-25m.zip

--2024-11-09 11:22:53--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  20.3MB/s    in 14s     

2024-11-09 11:23:07 (18.3 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [3]:
# Criar a sessao do Spark
from pyspark.sql import SparkSession
spark = SparkSession \
            .builder \
            .master("local[*]") \
            .appName("Michel_DF") \
            .getOrCreate()

In [4]:
df = spark.read.csv('/content/ml-25m/ratings.csv', header=True)

In [7]:
df.take(54)

[Row(userId='1', movieId='296', rating='5.0', timestamp='1147880044'),
 Row(userId='1', movieId='306', rating='3.5', timestamp='1147868817'),
 Row(userId='1', movieId='307', rating='5.0', timestamp='1147868828'),
 Row(userId='1', movieId='665', rating='5.0', timestamp='1147878820'),
 Row(userId='1', movieId='899', rating='3.5', timestamp='1147868510')]

In [11]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, BooleanType, DoubleType, DoubleType, IntegerType

In [12]:
colunas = [('user_id',IntegerType()),
           ('movie_id',IntegerType()),
           ('rating', DoubleType()),
           ('timestamp',StringType())]

In [13]:
colunas

[('user_id', IntegerType()),
 ('movie_id', IntegerType()),
 ('rating', DoubleType()),
 ('timestamp', StringType())]

In [16]:
schema = StructType([StructField(colName,colType, True) for colName, colType in colunas])
schema

StructType([StructField('user_id', IntegerType(), True), StructField('movie_id', IntegerType(), True), StructField('rating', DoubleType(), True), StructField('timestamp', StringType(), True)])

In [17]:
df = spark.read.csv('/content/ml-25m/ratings.csv', header=True, schema = schema )

In [19]:
df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- movie_id: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: string (nullable = true)



In [25]:
print("Número atual de partições:", df.rdd.getNumPartitions())


Número atual de partições: 6
Configurações do Spark: [('spark.app.name', 'Michel_DF'), ('spark.app.submitTime', '1731151492639'), ('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDire

Agrupar os dados por movieID para contar quantas avaliações cada filme teve

In [26]:
%%time
df.groupBy('movie_id').count().show()

+--------+-----+
|movie_id|count|
+--------+-----+
|    1088|11935|
|    1580|40308|
|    3175|14659|
|   44022| 4833|
|  175197|  610|
|    1645|13496|
|     471|10631|
|    3794|  763|
|    8638| 4832|
|   33722|  181|
|    2142| 2179|
|    2366| 6358|
|    6658|  740|
|    1959| 4763|
|    6620| 3865|
|   54190| 2351|
|    3918| 1371|
|   68135| 2676|
|    1342| 3546|
|    1591| 5351|
+--------+-----+
only showing top 20 rows

CPU times: user 196 ms, sys: 22.8 ms, total: 219 ms
Wall time: 36.5 s


In [31]:
df.groupBy('movie_id').count().explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[movie_id#52], functions=[count(1)])
   +- Exchange hashpartitioning(movie_id#52, 200), ENSURE_REQUIREMENTS, [plan_id=242]
      +- HashAggregate(keys=[movie_id#52], functions=[partial_count(1)])
         +- InMemoryTableScan [movie_id#52]
               +- InMemoryRelation [user_id#51, movie_id#52, rating#53, timestamp#54], StorageLevel(disk, memory, deserialized, 1 replicas)
                     +- FileScan csv [user_id#51,movie_id#52,rating#53,timestamp#54] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/ml-25m/ratings.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_id:int,movie_id:int,rating:double,timestamp:string>




In [30]:
df.createOrReplaceTempView("tabela_filmes")
df.show(5)

+-------+--------+------+----------+
|user_id|movie_id|rating| timestamp|
+-------+--------+------+----------+
|      1|     296|   5.0|1147880044|
|      1|     306|   3.5|1147868817|
|      1|     307|   5.0|1147868828|
|      1|     665|   5.0|1147878820|
|      1|     899|   3.5|1147868510|
+-------+--------+------+----------+
only showing top 5 rows



In [32]:
query = spark.sql("SELECT * FROM tabela_filmes LIMIT 5")
query.show()

+-------+--------+------+----------+
|user_id|movie_id|rating| timestamp|
+-------+--------+------+----------+
|      1|     296|   5.0|1147880044|
|      1|     306|   3.5|1147868817|
|      1|     307|   5.0|1147868828|
|      1|     665|   5.0|1147878820|
|      1|     899|   3.5|1147868510|
+-------+--------+------+----------+



In [34]:
query.explain(mode="formatted")

== Physical Plan ==
AdaptiveSparkPlan (5)
+- CollectLimit (4)
   +- InMemoryTableScan (1)
         +- InMemoryRelation (2)
               +- Scan csv  (3)


(1) InMemoryTableScan
Output [4]: [user_id#51, movie_id#52, rating#53, timestamp#54]
Arguments: [user_id#51, movie_id#52, rating#53, timestamp#54]

(2) InMemoryRelation
Arguments: [user_id#51, movie_id#52, rating#53, timestamp#54], CachedRDDBuilder(org.apache.spark.sql.execution.columnar.DefaultCachedBatchSerializer@11a36655,StorageLevel(disk, memory, deserialized, 1 replicas),FileScan csv [user_id#51,movie_id#52,rating#53,timestamp#54] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/ml-25m/ratings.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_id:int,movie_id:int,rating:double,timestamp:string>
,None)

(3) Scan csv 
Output [4]: [user_id#51, movie_id#52, rating#53, timestamp#54]
Batched: false
Location: InMemoryFileIndex [file:/content/ml-25m/ratings.csv]
Read

In [38]:
query = spark.sql("SELECT movie_id, COUNT(*) AS count FROM tabela_filmes WHERE rating = 5 GROUP BY movie_id ORDER BY 2 DESC")
query.show()

+--------+-----+
|movie_id|count|
+--------+-----+
|     318|39553|
|     296|32169|
|     356|25918|
|     260|25804|
|    2571|25482|
|     527|24853|
|     593|24801|
|     858|24418|
|      50|21585|
|    2959|21486|
|    1196|20893|
|     110|18785|
|    4993|18611|
|    1198|17725|
|    2858|17057|
|    7153|16779|
|     608|16057|
|    5952|15858|
|    1210|15665|
|     589|14774|
+--------+-----+
only showing top 20 rows



In [ ]:
df = spark.read.options(header=True).csv('supermercado.csv')
df.show(5)

+-----------------------------------+
|PRODUTO;QUANTIDADE;PRECO UNIT. (R$)|
+-----------------------------------+
|                Achocolatado;2;5.89|
|                      Acucar;3;2.03|
|                       Agua;10;1.89|
|                      Alface;1;2.99|
|                      Arroz;4;14.87|
+-----------------------------------+
only showing top 5 rows



In [ ]:
df = spark.read.options(header=True, delimiter=";").csv('supermercado.csv')
df.show(5)

+------------+----------+----------------+
|     PRODUTO|QUANTIDADE|PRECO UNIT. (R$)|
+------------+----------+----------------+
|Achocolatado|         2|            5.89|
|      Acucar|         3|            2.03|
|        Agua|        10|            1.89|
|      Alface|         1|            2.99|
|       Arroz|         4|           14.87|
+------------+----------+----------------+
only showing top 5 rows



In [ ]:
df = spark.read.options(header=True, delimiter=";", inferSchema=True).csv('supermercado.csv')
df.show(5)

+------------+----------+----------------+
|     PRODUTO|QUANTIDADE|PRECO UNIT. (R$)|
+------------+----------+----------------+
|Achocolatado|       2.0|            5.89|
|      Acucar|       3.0|            2.03|
|        Agua|      10.0|            1.89|
|      Alface|       1.0|            2.99|
|       Arroz|       4.0|           14.87|
+------------+----------+----------------+
only showing top 5 rows



In [ ]:
df.schema

StructType([StructField('PRODUTO', StringType(), True), StructField('QUANTIDADE', DoubleType(), True), StructField('PRECO UNIT. (R$)', DoubleType(), True)])

In [ ]:
df = spark.read.options(header=True, delimiter=";", inferSchema=True).csv('supermercado.csv').toDF("produto","quantidade","preco_UN")
df.show(5)


+------------+----------+--------+
|     produto|quantidade|preco_UN|
+------------+----------+--------+
|Achocolatado|       2.0|    5.89|
|      Acucar|       3.0|    2.03|
|        Agua|      10.0|    1.89|
|      Alface|       1.0|    2.99|
|       Arroz|       4.0|   14.87|
+------------+----------+--------+
only showing top 5 rows



In [ ]:
import pyspark.sql.functions as sf
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, DoubleType, TimestampType, IntegerType, StructType, StructField, BooleanType

In [ ]:
spark_udf = udf(lambda x: x>5,returnType=BooleanType())

In [ ]:
df.withColumn("maior_5",spark_udf(sf.col("quantidade"))).show()

+-------------------+----------+--------+-------+
|            produto|quantidade|preco_UN|maior_5|
+-------------------+----------+--------+-------+
|       Achocolatado|       2.0|    5.89|  false|
|             Acucar|       3.0|    2.03|  false|
|               Agua|      10.0|    1.89|   true|
|             Alface|       1.0|    2.99|  false|
|              Arroz|       4.0|   14.87|  false|
|               Atum|       3.0|    7.45|  false|
|             Azeite|       2.0|    17.8|  false|
|           Azeitona|       2.0|    8.04|  false|
|             Batata|       2.8|    4.99|  false|
|       Batata palha|       5.0|    5.99|  false|
|               Cafe|       3.0|     9.7|  false|
|             Cebola|      0.76|    3.45|  false|
|            Cenoura|      1.98|    3.99|  false|
|                Cha|       2.0|    3.67|  false|
|          Chocolate|       1.0|     9.9|  false|
|        Coco ralado|       2.0|    3.78|  false|
|     Creme de Leite|       5.0|    2.46|  false|


In [ ]:
is_integer_udf = udf(lambda x: x.is_integer(), returnType=BooleanType())

In [ ]:
df.withColumn("inteiro",is_integer_udf(sf.col("preco_UN"))).show()

+-------------------+----------+--------+-------+
|            produto|quantidade|preco_UN|inteiro|
+-------------------+----------+--------+-------+
|       Achocolatado|       2.0|    5.89|  false|
|             Acucar|       3.0|    2.03|  false|
|               Agua|      10.0|    1.89|  false|
|             Alface|       1.0|    2.99|  false|
|              Arroz|       4.0|   14.87|  false|
|               Atum|       3.0|    7.45|  false|
|             Azeite|       2.0|    17.8|  false|
|           Azeitona|       2.0|    8.04|  false|
|             Batata|       2.8|    4.99|  false|
|       Batata palha|       5.0|    5.99|  false|
|               Cafe|       3.0|     9.7|  false|
|             Cebola|      0.76|    3.45|  false|
|            Cenoura|      1.98|    3.99|  false|
|                Cha|       2.0|    3.67|  false|
|          Chocolate|       1.0|     9.9|  false|
|        Coco ralado|       2.0|    3.78|  false|
|     Creme de Leite|       5.0|    2.46|  false|


In [ ]:
df.withColumn("inteiro",is_integer_udf(sf.col("quantidade"))).show()

+-------------------+----------+--------+-------+
|            produto|quantidade|preco_UN|inteiro|
+-------------------+----------+--------+-------+
|       Achocolatado|       2.0|    5.89|   true|
|             Acucar|       3.0|    2.03|   true|
|               Agua|      10.0|    1.89|   true|
|             Alface|       1.0|    2.99|   true|
|              Arroz|       4.0|   14.87|   true|
|               Atum|       3.0|    7.45|   true|
|             Azeite|       2.0|    17.8|   true|
|           Azeitona|       2.0|    8.04|   true|
|             Batata|       2.8|    4.99|  false|
|       Batata palha|       5.0|    5.99|   true|
|               Cafe|       3.0|     9.7|   true|
|             Cebola|      0.76|    3.45|  false|
|            Cenoura|      1.98|    3.99|  false|
|                Cha|       2.0|    3.67|   true|
|          Chocolate|       1.0|     9.9|   true|
|        Coco ralado|       2.0|    3.78|   true|
|     Creme de Leite|       5.0|    2.46|   true|


In [ ]:
df2 = df.withColumn("qtde_arrumada",sf.when(is_integer_udf(sf.col("quantidade")),sf.col("quantidade")).otherwise(1))
df2.show()

+-------------------+----------+--------+-------------+
|            produto|quantidade|preco_UN|qtde_arrumada|
+-------------------+----------+--------+-------------+
|       Achocolatado|       2.0|    5.89|          2.0|
|             Acucar|       3.0|    2.03|          3.0|
|               Agua|      10.0|    1.89|         10.0|
|             Alface|       1.0|    2.99|          1.0|
|              Arroz|       4.0|   14.87|          4.0|
|               Atum|       3.0|    7.45|          3.0|
|             Azeite|       2.0|    17.8|          2.0|
|           Azeitona|       2.0|    8.04|          2.0|
|             Batata|       2.8|    4.99|          1.0|
|       Batata palha|       5.0|    5.99|          5.0|
|               Cafe|       3.0|     9.7|          3.0|
|             Cebola|      0.76|    3.45|          1.0|
|            Cenoura|      1.98|    3.99|          1.0|
|                Cha|       2.0|    3.67|          2.0|
|          Chocolate|       1.0|     9.9|       

In [ ]:
df2.agg({"qtde_arrumada":"sum"}).show()

+------------------+
|sum(qtde_arrumada)|
+------------------+
|             122.0|
+------------------+



In [ ]:
# prompt: quero gerar uma coluna contendo o valor total da compra (quantidade X preço)

df3 = df2.withColumn("valor_total", sf.col("preco_UN") * sf.col("qtde_arrumada"))
df3.show()

+-------------------+----------+--------+-------------+------------------+
|            produto|quantidade|preco_UN|qtde_arrumada|       valor_total|
+-------------------+----------+--------+-------------+------------------+
|       Achocolatado|       2.0|    5.89|          2.0|             11.78|
|             Acucar|       3.0|    2.03|          3.0|              6.09|
|               Agua|      10.0|    1.89|         10.0|              18.9|
|             Alface|       1.0|    2.99|          1.0|              2.99|
|              Arroz|       4.0|   14.87|          4.0|             59.48|
|               Atum|       3.0|    7.45|          3.0|             22.35|
|             Azeite|       2.0|    17.8|          2.0|              35.6|
|           Azeitona|       2.0|    8.04|          2.0|             16.08|
|             Batata|       2.8|    4.99|          1.0|              4.99|
|       Batata palha|       5.0|    5.99|          5.0|29.950000000000003|
|               Cafe|    

In [ ]:
df3.agg({"valor_total":"sum"}).show()

+-----------------+
| sum(valor_total)|
+-----------------+
|798.8499999999998|
+-----------------+



In [ ]:
df3.sort(sf.col("valor_total").desc()).show()

+--------------+----------+--------+-------------+------------------+
|       produto|quantidade|preco_UN|qtde_arrumada|       valor_total|
+--------------+----------+--------+-------------+------------------+
|         Vinho|       2.0|    37.0|          2.0|              74.0|
|         Arroz|       4.0|   14.87|          4.0|             59.48|
|      Presunto|      0.35|    50.0|          1.0|              50.0|
|          Suco|      10.0|    4.97|         10.0|49.699999999999996|
|        Queijo|       0.4|    45.0|          1.0|              45.0|
|     Mortadela|       0.3|    40.0|          1.0|              40.0|
|        Geleia|       4.0|     8.9|          4.0|              35.6|
|        Azeite|       2.0|    17.8|          2.0|              35.6|
|  Refrigerante|       5.0|     6.0|          5.0|              30.0|
|  Batata palha|       5.0|    5.99|          5.0|29.950000000000003|
|          Cafe|       3.0|     9.7|          3.0|29.099999999999998|
|          Atum|    

In [ ]:
df = spark.read.csv('../10_dados/movie_lens/ratings.csv', header=True, schema = schema )

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
# Criar a sessao do Spark
from pyspark.sql import SparkSession
spark = SparkSession \
            .builder \
            .master("local[*]") \
            .appName("Michel_DF") \
            .getOrCreate()

In [ ]:
df = spark.read.csv('/content/ml-25m/ratings.csv', header=True)